<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Tetrabot%20/Vasiliy_Borisov/09_19/tetrabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загружаем библиотеки, подключаем гугл диск

In [ ]:
!pip install SpeechRecognition
!pip install fasttext
import requests
# from urllib.parse import urlencode
# import json
import gdown
import os
import zipfile
import numpy as np
import pandas as pd
from google.colab import drive
import fasttext
import speech_recognition as sr
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, concatenate, Input, CategoryEncoding, Normalization, Activation, Conv1D
from tensorflow.keras.optimizers import Adam, AdamW, RMSprop
from tensorflow.keras import utils
import tensorflow as tf


r = sr.Recognizer()
drive.mount('/content/gdrive')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=dfabf469a8a83f4a9c89f22043ffcef26c019334e793bbdd74a080522cc7f2e9
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Mounted at /content/gdrive


In [ ]:
ft = fasttext.load_model('/content/gdrive/MyDrive/AI_data/cc.ru.300.bin')


# Загружаем файл с разметкой

In [ ]:

x_train = []

y_train_group = np.array([])
y_train_action = np.array([])

df_razmetka = pd.read_excel('/content/gdrive/MyDrive/AI_data/razmetka.xlsx')
for idx, row in df_razmetka.iterrows():
  x_train.append(ft.get_word_vector(row.text))
y_train_group = df_razmetka['group'].to_numpy()
y_train_action = df_razmetka['action'].to_numpy()

In [ ]:
print((x_train[1]))

# Описываем и запускаем модель

In [ ]:
trainY_group = tf.keras.utils.to_categorical(y_train_group)

x_train = tf.stack(x_train)

model_group = Sequential()
model_group.add(Dense(len(x_train[0]), activation='relu'))
model_group.add(Dense(800, activation='relu'))
model_group.add(Dense(400, activation='relu'))
model_group.add(Dense(trainY_group[0].shape[0], activation='softmax'))
model_group.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model_group.fit(x_train, trainY_group, batch_size=128, epochs=150, verbose=1)

trainY_action = tf.keras.utils.to_categorical(y_train_action)

model_action = Sequential()
model_action.add(Dense(len(x_train[0]), activation='relu'))
model_action.add(Dense(800, activation='relu'))
model_action.add(Dense(400, activation='relu'))
model_action.add(Dense(trainY_action[0].shape[0], activation='softmax'))
model_action.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model_action.fit(x_train, trainY_action, batch_size=128, epochs=150, verbose=1)




Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 2.2985 - accuracy: 0.1071
Epoch 2/150
1/1 [==============================] - 0s 33ms/step - loss: 2.2297 - accuracy: 0.5833
Epoch 3/150
1/1 [==============================] - 0s 52ms/step - loss: 2.1528 - accuracy: 0.5833
Epoch 4/150
1/1 [==============================] - 0s 49ms/step - loss: 2.0524 - accuracy: 0.5833
Epoch 5/150
1/1 [==============================] - 0s 35ms/step - loss: 1.9217 - accuracy: 0.5833
Epoch 6/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7639 - accuracy: 0.5833
Epoch 7/150
1/1 [==============================] - 0s 35ms/step - loss: 1.6031 - accuracy: 0.5833
Epoch 8/150
1/1 [==============================] - 0s 35ms/step - loss: 1.4807 - accuracy: 0.5833
Epoch 9/150
1/1 [==============================] - 0s 42ms/step - loss: 1.4208 - accuracy: 0.5833
Epoch 10/150
1/1 [==============================] - 0s 45ms/step - loss: 1.3821 - accuracy: 0.5833
Epoch 11/150
1/1 [===

# Описываем функцию распознавани аудио при помощи recognize_google

In [ ]:

def recognize_file_def(AudioFile):
  AudioFile = sr.AudioFile(AudioFile)
  with AudioFile as source:
      audio = r.record(source)
      return r.recognize_google(audio, language="ru-RU")

# Перебираем файлы, распознаем текст

In [ ]:
df = pd.DataFrame(columns=["file", "text", "vector"])#, "text", "vector")

x_predict = []

def search_all_files(directory):
  for filename in os.listdir(directory):
    if "chunk" in filename:
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
          if os.path.getsize(f)<90000000:
            recognize_file = recognize_file_def(f)
            word_vector = ft.get_word_vector(recognize_file)
            x_predict.append(word_vector)
            # word_vector = ft.get_word_vector(recognize_file)
            df.loc[ len(df.index )] = [f, recognize_file, word_vector]
        else:
          search_all_files(f)

search_all_files('/content/gdrive/MyDrive/AI_data/golos_blizko')
df.to_excel("/content/gdrive/MyDrive/AI_data/df.xlsx")


In [ ]:
# (x_train[1][2])
x_predict = tf.stack(x_predict)

In [ ]:
print(x_predict)

tf.Tensor(
[[ 0.01934599 -0.01369245 -0.00029067 ...  0.00233922 -0.00719262
  -0.01895417]
 [ 0.01971263 -0.01710786  0.01421099 ...  0.00729356  0.00133919
  -0.0179143 ]
 [ 0.01272598 -0.02894953  0.00155694 ... -0.00242595 -0.02604934
  -0.02335784]
 ...
 [ 0.02442752 -0.01972329  0.01674293 ... -0.00585412 -0.02110018
  -0.03100809]
 [ 0.00353123 -0.02000297  0.00148613 ...  0.01097282 -0.04391326
  -0.02362892]
 [ 0.01311401 -0.02285752 -0.00102524 ... -0.00992145 -0.02438494
  -0.02608792]], shape=(97, 300), dtype=float32)


# Пропускаем распознанный текст  через нашу модель

In [ ]:
print((x_predict[0][0]))
print(x_train[0][0])


tf.Tensor(0.01934599, shape=(), dtype=float32)
tf.Tensor(0.030854637, shape=(), dtype=float32)


In [ ]:
print(len(x_predict[0]))
prediction
model.summary()
print(prediction[0])
(np.argmax(prediction[0]))

300
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               90300     
                                                                 
 dense_1 (Dense)             (None, 800)               240800    
                                                                 
 dense_2 (Dense)             (None, 400)               320400    
                                                                 
 dense_3 (Dense)             (None, 10)                4010      
                                                                 
Total params: 655510 (2.50 MB)
Trainable params: 655510 (2.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
[3.7744860e-07 9.5897216e-01 9.6652839e-05 1.7588292e-03 3.8480707e-06
 3.5315854e-06 6.7758887e-10 5.8939709e-10 9.5375075e-10 3.9164513e-02]


1

In [ ]:
len(x_predict[0])

300

In [ ]:
prediction_group = model_group.predict(x_predict)
prediction_action = model_action.predict(x_predict)
for index, row in df.iterrows():
  print(len(x_predict[index]))

  print(np.argmax(prediction_group[index]))
  print(np.argmax(prediction_action[index]))




4/4 [==============================] - 0s 4ms/step
300
1
1
300
2
4
300
1
5
300
2
3
300
3
1
300
3
1
300
3
1
300
9
1
300
1
4
300
1
6
300
1
2
300
1
1
300
1
1
300
1
1
300
1
3
300
1
1
300
1
4
300
1
4
300
1
3
300
1
3
300
1
3
300
1
4
300
1
6
300
1
6
300
1
6
300
5
1
300
5
1
300
9
1
300
1
4
300
1
4
300
1
3
300
1
3
300
9
1
300
9
1
300
9
1
300
1
6
300
4
6
300
4
6
300
1
5
300
1
5
300
1
5
300
1
2
300
1
2
300
1
2
300
1
1
300
1
1
300
1
1
300
5
6
300
5
6
300
0
0
300
0
0
300
0
0
300
2
5
300
2
5
300
2
1
300
1
6
300
1
6
300
1
6
300
1
3
300
1
3
300
1
4
300
1
4
300
1
5
300
0
0
300
0
0
300
0
0
300
1
1
300
1
1
300
1
2
300
1
2
300
1
2
300
3
6
300
3
6
300
3
6
300
4
1
300
4
1
300
4
1
300
2
2
300
2
2
300
2
6
300
2
6
300
1
6
300
1
6
300
1
1
300
1
1
300
1
1
300
1
1
300
1
1
300
1
1
300
1
1
300
1
4
300
1
6
300
1
6
300
1
6
300
1
6
300
1
6
300
1
6
